# Projet NoSQL 
## Naomi Girard

### Choix de la base de données

J'ai décidé de travailler avec MongoDB parce qu'il s'agit d'une base de données orientée document. Les documents comprennent des ensembles de clé-valeur. Elle permet de charger très facilement des fichiers JSON. Un élement d'un JSON correspond à un document dans une base Mongo. Mongo n'exige pas de structure pré-définie de ses document et permet donc une grande flexibilité dans les données. 

Un autre avantage qui a fait basculer mon choix vers MongoDB est sa très bonne compatibilté avec Python via le package "pymongo". Il est donc facile de faire un pré-processing sur les données, les charger et exécuter les requêtes via uniquement Python. De plus, je peux combiner python à un notebook rendant l'affichage et la lecture des requêtes plus esthétiques que sur un terminal. 


### Ma définition du parfait emplacement à New York

Pour moi vivre à New York, ne signifie pas vivre à Manhattan en plein Time Square. Je recherche avant tout un endroit proche d'une piscine, ayant pour habitude de nager plusieurs fois par semaine. Il ne faut pas qu'elle soit trop loin, sinon je serais découragée par la distance et n'irai pas. Je cherche également à éviter la malbouffe américaine et veut être à proximité de primeurs/marchés pour avoir accès à de bons produits. 
De plus, si je peux de temps en temps me rendre dans un restaurant français lorsque le mal du pays se fera ressentir, ça ne rendra l'emplacement que plus parfait. Enfin, l'accès au métro est primordial mais je suis prête à sacrifier un peu de temps de trajet pour me rendre à Manhattan, là où la ville grouille. 

### Import et connexion au serveur mongo

In [1]:
import json
import os
import pymongo
import pandas as pd
import numpy as np
from uszipcode import ZipcodeSearchEngine

cnx_string = "mongodb://localhost:1234"
client = pymongo.MongoClient(cnx_string)

In [2]:
client.drop_database('nyc')
client.database_names()
db = client.nyc

In [3]:
db.collection_names()

['french', 'subway', 'pools2', 'restaurants', 'pools', 'subway2']

## 1. Piscines

### 1.1 Chargement des données dans la base nyc

In [79]:
def insert_data(path, col):
    data = open(path, 'r')
    parsed = json.loads(data.read())
    for doc in parsed: 
        db[col].insert_one(doc)

### Chargement de la base pools alternative avec un léger pre-processing

Les données initiales comportent les coordonnées de chaque piscine sous forme de multipolygon GeoJson. Ce type d'objet est plutôt difficile à manipuler et je n'ai pas besoin d'autant de détails sur les positions des piscines à New York. J'ai donc résumé cette information par un semblant de barycentre du polygone, en prenant la moyenne des latitudes et longitudes. Enfin, la base ne donne pas le code postal qui pourra m'être utile pour le croiser avec d'autres bases de données. J'ai pour cela utilisé une library python "uszipcode" qui permet de trouver, entre autres, le code postal relatif à une position géographique.  

In [24]:
def insert_data_pool(path, col):
    data = open(path, 'r')
    parsed = json.loads(data.read())
    search = ZipcodeSearchEngine()
    for doc in parsed: 
        a = doc['the_geom']['coordinates']
        a = sum(sum(a, []), [])
        df = pd.DataFrame(a, columns = ['long', 'lat'])
        coord = (np.mean(df['lat']), np.mean(df['long']))
        doc['zipcode'] = search.by_coordinate(coord[0], coord[1], radius=3, returns= 1)[0]["Zipcode"]
        #retourne le code postal le plus proche dans un rayon de 3 miles (5 km), par rapport aux coordonnées fournies en paremètres
        doc['average_coord'] = coord
        db[col].insert_one(doc)

In [25]:
insert_data_pool("pools.json", "pools")

In [26]:
db.pools.find_one()

{'_id': ObjectId('5a7f3f68a4629b27c96eaef5'),
 'average_coord': [40.83928337337758, -73.8984379901158],
 'borough': 'X',
 'councildis': '15',
 'gispropnum': 'X010',
 'location': 'Outdoor',
 'name': 'Crotona Pool',
 'objectid': '205',
 'pooltype': 'Diving',
 'shape_star': '7173.03729248',
 'shape_stle': '326.892595219',
 'the_geom': {'coordinates': [[[[-73.89854807978361, 40.83945171469163],
     [-73.89818724520086, 40.83933438115879],
     [-73.89817715050593, 40.83931558074005],
     [-73.89818279386012, 40.839302773494],
     [-73.89818956243367, 40.839289968141415],
     [-73.89819745254967, 40.83927801744655],
     [-73.89821773105048, 40.83925582736127],
     [-73.89822899302501, 40.83924558787669],
     [-73.89824250295007, 40.83923620314201],
     [-73.8982560115616, 40.83922767477428],
     [-73.89827064408748, 40.83922000016408],
     [-73.89828639934072, 40.83921318021037],
     [-73.89830215316371, 40.83920806939077],
     [-73.89831903102011, 40.839202959559934],
     [-73

### 1.2 Requêtes

On vient d'ajouter une base "pools" cataloguant les piscines de NYC. Visiblement il en existe de différents types:


In [27]:
pipeline = [{"$group" : {"_id": "$pooltype"}}]
import pprint
pprint.pprint(list(db.pools.aggregate(pipeline)))

[{'_id': 'Mini'},
 {'_id': 'Diving'},
 {'_id': 'Wading'},
 {'_id': 'Intermediate'},
 {'_id': 'Olympic'}]


Seulement les piscines de type "Intermediate" et "Olympic" m'interessent pour la suite. 

In [113]:
pprint.pprint(list(
    db.pools.find({"$and": [{"location" : "Indoor"},   
                            {"$or" : [
                                    {"pooltype" : "Intermediate"}, {"pooltype" : "Olympic"}
                            ]}]}
        ,{"the_geom":0})
), width = 1)

[{'_id': ObjectId('5a7f3f68a4629b27c96eaef6'),
  'average_coord': [40.81143738049233,
                    -73.91413763379475],
  'borough': 'X',
  'councildis': '17',
  'gispropnum': 'X045',
  'location': 'Indoor',
  'name': 'St. '
          "Mary's "
          'Pool',
  'objectid': '206',
  'pooltype': 'Intermediate',
  'shape_star': '2807.98722839',
  'shape_stle': '221.999701843',
  'zipcode': '10455'},
 {'_id': ObjectId('5a7f3f68a4629b27c96eaef7'),
  'average_coord': [40.687281611315214,
                    -73.76954835782503],
  'borough': 'Q',
  'councildis': '27',
  'gispropnum': 'Q448',
  'location': 'Indoor',
  'name': 'Roy '
          'Wilkins '
          'Pool',
  'objectid': '207',
  'pooltype': 'Intermediate',
  'shape_star': '2249.99740601',
  'shape_stle': '209.999708732',
  'zipcode': '11434'},
 {'_id': ObjectId('5a7f3f68a4629b27c96eaefd'),
  'average_coord': [40.73617475654855,
                    -73.9756943885272],
  'borough': 'M',
  'councildis': '4',
  'gispropnum

Pour faire un premier tri sur les quartiers contenant le plus de piscines, je cherche le nombre de piscines Intermediate ou Olympic (pour les piscines intérieures) par quartier par ordre décroissant : 


In [29]:
pipeline = [{"$match":{"$and": [{"location" : "Indoor"},   
                            {"$or" : [
                                    {"pooltype" : "Intermediate"}, {"pooltype" : "Olympic"}
                            ]}]}},
    {"$group": {"_id" : "$borough", "total": {"$sum":1}}},
    {"$sort": {"total":-1}}]
pprint.pprint(list(db.pools.aggregate(pipeline)))

[{'_id': 'M', 'total': 14},
 {'_id': 'B', 'total': 6},
 {'_id': 'Q', 'total': 4},
 {'_id': 'X', 'total': 2}]


J'aimerais affiner cet affichage en ajoutant le nombre de piscine Intermediate ou Olympic par quartier : 

In [114]:
pipeline = [{"$match":{"$and": [{"location" : "Indoor"},   
                            {"$or" : [
                                    {"pooltype" : "Intermediate"}, {"pooltype" : "Olympic"}
                            ]}]}},
{"$group": {
    "_id": {"borough": "$borough", "pooltype": "$pooltype"},
    "poolcount": {"$sum":1}}},
    {"$group" : {"_id" : "$_id.borough", 
        "ptype" : { "$push": {
            "pooltype" : "$_id.pooltype",
            "count pooltype per borough" : "$poolcount"
            }},
           "total pools per borough": {"$sum": "$poolcount"} }}]
pprint.pprint(list(db.pools.aggregate(pipeline)), width = 1)

[{'_id': 'B',
  'ptype': [{'count pooltype per borough': 6,
             'pooltype': 'Intermediate'}],
  'total pools per borough': 6},
 {'_id': 'M',
  'ptype': [{'count pooltype per borough': 14,
             'pooltype': 'Intermediate'}],
  'total pools per borough': 14},
 {'_id': 'X',
  'ptype': [{'count pooltype per borough': 2,
             'pooltype': 'Intermediate'}],
  'total pools per borough': 2},
 {'_id': 'Q',
  'ptype': [{'count pooltype per borough': 2,
             'pooltype': 'Olympic'},
            {'count pooltype per borough': 2,
             'pooltype': 'Intermediate'}],
  'total pools per borough': 4}]


Je ferai mon choix entre Manhattan et Brooklyn. L'objectif serait pour moi d'affiner ma recherche vers un code postal. Aux Etats-Unis, un code postal représente une zone assez réduite et me donnera une idée assez précise de l'endroit où je souhaite vivre à New York. Ainsi, je regarde quels sont les codes postaux de Manhattan et Brooklyn ayant au moins une piscine : 

In [115]:
pipeline = [{"$match":{"$and": [{"location" : "Indoor"},   
                            {"$or" : [
                                    {"pooltype" : "Intermediate"}, {"pooltype" : "Olympic"}]},
                               {"$or" : [
                                    {"borough" : "M"}, {"borough" : "B"}]}]}},
{"$group": {
    "_id": {"borough": "$borough", "zipcode": "$zipcode"},
    "poolcount": {"$sum":1}}},
           {"$sort": {"poolcount":-1}}]
pprint.pprint(list(db.pools.aggregate(pipeline)), width = 1)

[{'_id': {'borough': 'M',
          'zipcode': '10022'},
  'poolcount': 4},
 {'_id': {'borough': 'B',
          'zipcode': '11212'},
  'poolcount': 2},
 {'_id': {'borough': 'M',
          'zipcode': '10012'},
  'poolcount': 2},
 {'_id': {'borough': 'B',
          'zipcode': '11211'},
  'poolcount': 2},
 {'_id': {'borough': 'M',
          'zipcode': '10037'},
  'poolcount': 2},
 {'_id': {'borough': 'M',
          'zipcode': '10010'},
  'poolcount': 2},
 {'_id': {'borough': 'B',
          'zipcode': '11213'},
  'poolcount': 2},
 {'_id': {'borough': 'M',
          'zipcode': '10199'},
  'poolcount': 2},
 {'_id': {'borough': 'M',
          'zipcode': '10019'},
  'poolcount': 2}]


A première vue, le code postal 10022 à Manhattan serait le plus optimal puisque j'aurais le choix entre 4 piscines. Une me suffirait déjà beaucoup je ne vais donc pas m'arrêter ici et je récupère dans une liste les codes postaux, ils m'aideront à affiner mon choix avec les autres critères.

In [34]:
reszip = list(db.pools.aggregate(pipeline))
zipb = []
zipm = []
for elem in reszip : 
    if elem['_id']['borough'] == "B": 
        zipb.append(elem['_id']['zipcode'])
    else : 
        zipm.append(elem['_id']['zipcode'])

## 2. Farmer Markets

Un autre critère essentiel pour moi est la facilité à faire ses courses. Les américains sont connus pour leur malbouffe, j'aimerais y échapper et avoir à proximité un marché/primeur.

### 2.1 Chargement des données en base

In [116]:
json_markets = open("markets.json", 'r')
parsed_markets = json.loads(json_markets.read())

for item in parsed_markets:
    db.markets.insert_one(item)

print(db.markets.find_one())

{'_id': ObjectId('5a809ab4a4629b16115abd79'), 'facilityaddinfo': "Spend $5 in EBT and get a $2 Health Buck coupon.<br><br><b>Program Website:</b> <a  href=''http://www.grownyc.org/youthmarket'' target=''_blank''>http://www.grownyc.org/youthmarket</a>", 'facilitycity': 'Bronx', 'facilityname': 'Riverdale Youthmarket', 'facilitystate': 'NY', 'facilitystreetname': '256th St & Mosholu Ave', 'facilityzipcode': '10471', 'latitude': '41', 'longitude': '-74'}


In [10]:
pipeline = [
    {"$group": {"_id" : "$facilitycity", "total": {"$sum":1}}},
    {"$sort": {"total":-1}}
]
pprint.pprint(list(db.markets.aggregate(pipeline)))

[{'_id': 'Manhattan', 'total': 36},
 {'_id': 'Brooklyn', 'total': 34},
 {'_id': 'Bronx', 'total': 18},
 {'_id': 'Queens', 'total': 12},
 {'_id': None, 'total': 1},
 {'_id': 'Staten Island', 'total': 1}]


Encore une fois, je retiens Manhattan et Brooklyn qui ont de loin le plus de farmer markets. 

Requete qui retourne le nombre de farmers markets par quartier et par code postal, triée par quartier et ensuite par zipcode. 

(Le total par quartier ne correspond pas à celui de la requête précendente, je suppose que l'information du zipcode est parfois manquante)

In [118]:
pipeline = [
    {"$match":{"$or" : [
    {"facilitycity" : "Manhattan"}, {"facilitycity" : "Brooklyn"}
    ]}},
    {"$group": {"_id" : {"borough": "$facilitycity", "zipcode" : "$facilityzipcode"}, 
                "count": {"$sum":1}}},
    {"$group" : {"_id" : "$_id.borough", 
        "zipcode2" : { "$push": {
            "zipcode" : "$_id.zipcode",
            "per_zipcode" : "$count"
            }},
           "total markets per borough": {"$sum": "$count"}}},
    {"$unwind": "$zipcode2"},
    {"$sort" : {"total markets per borough": -1, "zipcode2.per_zipcode": -1}},
    {"$group": {"_id" : {"borough": "$_id"}, "total per borough": {"$sum":1},"total per zipcode": {"$push": "$zipcode2"}}},
    {"$sort": {"total per borough": -1}}
]
pprint.pprint(list(db.markets.aggregate(pipeline)), width = 1)

[{'_id': {'borough': 'Manhattan'},
  'total per borough': 26,
  'total per zipcode': [{'per_zipcode': 6,
                         'zipcode': '10029'},
                        {'per_zipcode': 4,
                         'zipcode': '10026'},
                        {'per_zipcode': 4,
                         'zipcode': '10003'},
                        {'per_zipcode': 4,
                         'zipcode': '10011'},
                        {'per_zipcode': 4,
                         'zipcode': '10019'},
                        {'per_zipcode': 4,
                         'zipcode': '10031'},
                        {'per_zipcode': 4,
                         'zipcode': '10004'},
                        {'per_zipcode': 4,
                         'zipcode': '10009'},
                        {'per_zipcode': 4,
                         'zipcode': '10027'},
                        {'per_zipcode': 2,
                         'zipcode': '10282'},
                        {'per_zipcode': 2,
     

## Restaurants

Autre critère essentiel, les restaurants. La base de données trouvée sur le site OpenDataNYC est très lourde j'ai donc fait un léger filtre sur le site avant de l'exporter (en ne gardant que les quartiers de Manhattan et Brooklyn). 

In [ ]:
def insert_data_resto(path, col):
    db[col].drop()
    json_data = open(path, 'r')
    parsed = json.loads(json_data.read())
    for doc in parsed['data']:
        k = {}
        k['CAMIS'] = doc[8]
        k['name'] = doc[9]
        k['borough'] = doc[10]
        k['building'] = doc[11]
        k['street'] = doc[12]
        k['zipcode'] = doc[13]
        k['cuisine_description'] = doc[14]
        db[col].insert_one(k)
    del json_data
    del parsed

L'import de la base peut prendre un peu de temps (environ une minute sur ma machine)!

In [ ]:
from time import time
st = time()
insert_data_resto("resto.json", "resto")
en = time()
print('exec time', en-st)

In [11]:
db.resto.find_one()

{'CAMIS': '41453657',
 '_id': ObjectId('5a808dbda4629b161157e996'),
 'borough': 'MANHATTAN',
 'building': '250',
 'cuisine_description': 'Sandwiches',
 'name': 'SUBWAY',
 'street': 'BROADWAY',
 'zipcode': '10007'}

In [13]:
pipeline = [{"$group" : {"_id": "$cuisine_description"}}]
print(list(db.resto.aggregate(pipeline)))

[{'_id': 'Czech'}, {'_id': 'Chilean'}, {'_id': 'Filipino'}, {'_id': 'Egyptian'}, {'_id': 'Portuguese'}, {'_id': 'Hotdogs/Pretzels'}, {'_id': 'Hotdogs'}, {'_id': 'Pancakes/Waffles'}, {'_id': 'Southwestern'}, {'_id': 'Brazilian'}, {'_id': 'Cajun'}, {'_id': 'Pakistani'}, {'_id': 'Chinese/Cuban'}, {'_id': 'Middle Eastern'}, {'_id': 'Armenian'}, {'_id': 'English'}, {'_id': 'Soul Food'}, {'_id': 'Peruvian'}, {'_id': 'Soups & Sandwiches'}, {'_id': 'Ethiopian'}, {'_id': 'Eastern European'}, {'_id': 'Moroccan'}, {'_id': 'Vegetarian'}, {'_id': 'Korean'}, {'_id': 'Pizza'}, {'_id': 'Chinese/Japanese'}, {'_id': 'Tapas'}, {'_id': 'Nuts/Confectionary'}, {'_id': 'CafÃ©/Coffee/Tea'}, {'_id': 'Juice, Smoothies, Fruit Salads'}, {'_id': 'Steak'}, {'_id': 'Sandwiches/Salads/Mixed Buffet'}, {'_id': 'Basque'}, {'_id': 'Ice Cream, Gelato, Yogurt, Ices'}, {'_id': 'Italian'}, {'_id': 'Caribbean'}, {'_id': 'Thai'}, {'_id': 'Jewish/Kosher'}, {'_id': 'Pizza/Italian'}, {'_id': 'Sandwiches'}, {'_id': 'Asian'}, {'_id

New york est connu pour sa diversité, en voici une preuve avec tous les types de cuisines disposibles. 
Pour faciliter la lecture des requêtes suivantes et mon choix je vais éliminer quelques types de cuisines. J'ai choisi d'afficher les types de cuisines ayant au moins 2500 restaurants à New York.

In [18]:
import pprint
pipeline = [
    {"$group": {"_id" : "$cuisine_description", "total": {"$sum":1}}},
    { "$match" : { "total" : { "$gt": 2500 } } },
    {"$sort": {"total":-1}}
]
pprint.pprint(list(db.resto.aggregate(pipeline)))

[{'_id': 'Chinese', 'total': 22839},
 {'_id': 'CafÃ©/Coffee/Tea', 'total': 13231},
 {'_id': 'Italian', 'total': 12693},
 {'_id': 'Japanese', 'total': 11174},
 {'_id': 'Pizza', 'total': 10000},
 {'_id': 'Mexican', 'total': 9394},
 {'_id': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
  'total': 7511},
 {'_id': 'Caribbean', 'total': 7470},
 {'_id': 'Bakery', 'total': 6775},
 {'_id': 'Spanish', 'total': 4758},
 {'_id': 'French', 'total': 4625},
 {'_id': 'Pizza/Italian', 'total': 4392},
 {'_id': 'Jewish/Kosher', 'total': 4275},
 {'_id': 'Indian', 'total': 3886},
 {'_id': 'Asian', 'total': 3759},
 {'_id': 'Thai', 'total': 3715},
 {'_id': 'Delicatessen', 'total': 3705},
 {'_id': 'Chicken', 'total': 3308},
 {'_id': 'Mediterranean', 'total': 2849},
 {'_id': 'Sandwiches', 'total': 2838},
 {'_id': 'Hamburgers', 'total': 2822},
 {'_id': 'Donuts', 'total': 2655}]


Je voudrais être relativement proche d'un restaurant français, lorsque le mal du pays se fera sentir. 
Je veux ainsi regarder les codes postaux ayant des piscines et des restaurants français. 

In [38]:
pipeline = [
    { "$match" : { "cuisine_description" : "French", "zipcode": {"$in": zipb+zipb}} },
    {"$group": {"_id" : {"borough": "$borough", "zipcode": "$zipcode"}, 
                "total": {"$sum":1}}},
    {"$sort": {"total":-1}}
]
pprint.pprint(list(db.resto.aggregate(pipeline)))

[{'_id': {'borough': 'BROOKLYN', 'zipcode': '11211'}, 'total': 129}]


Et en plus, c'est un quartier riche en restaurants et pas que français : 

In [120]:
pipeline = [
    { "$match" : {"zipcode": "11211" }},
    {"$group": {"_id" : {"cuisine_description": "$cuisine_description"}, 
                "total": {"$sum":1}}},
    {"$sort": {"total":-1}}
]
pprint.pprint(list(db.resto.aggregate(pipeline)))

[{'_id': {'cuisine_description': 'CafÃ©/Coffee/Tea'}, 'total': 351},
 {'_id': {'cuisine_description': 'Latin (Cuban, Dominican, Puerto Rican, South '
                                 '& Central American)'},
  'total': 282},
 {'_id': {'cuisine_description': 'Pizza'}, 'total': 255},
 {'_id': {'cuisine_description': 'Jewish/Kosher'}, 'total': 249},
 {'_id': {'cuisine_description': 'Chinese'}, 'total': 237},
 {'_id': {'cuisine_description': 'Japanese'}, 'total': 182},
 {'_id': {'cuisine_description': 'Mexican'}, 'total': 182},
 {'_id': {'cuisine_description': 'Italian'}, 'total': 164},
 {'_id': {'cuisine_description': 'French'}, 'total': 129},
 {'_id': {'cuisine_description': 'Bakery'}, 'total': 123},
 {'_id': {'cuisine_description': 'Thai'}, 'total': 104},
 {'_id': {'cuisine_description': 'Vegetarian'}, 'total': 88},
 {'_id': {'cuisine_description': 'Pizza/Italian'}, 'total': 84},
 {'_id': {'cuisine_description': 'Tex-Mex'}, 'total': 80},
 {'_id': {'cuisine_description': 'Indian'}, 'total

Je veux maintenant verifier si des farmer markets sont relativement proches de ce code postal:

In [47]:
list(db.markets.find({"facilityzipcode": '11211'}))

[]

Etant donné qu'il n'y a aucun primeur dans ce zipcode, je vais essayer de voir si il y en a dans les codes postaux à proximité. J'utilise pour cela le package python "uszipcode". J'y fais une recherche pour ce code postal, et conserve les coordonnées. Je fais ensuite une seconde recherche pour avoir des codes postaux avoisinant ces coordonées dans un rayon de 3miles (environ 5 kms).

In [58]:
search = ZipcodeSearchEngine()
lat = search.by_zipcode("11211")['Latitude']
lon = search.by_zipcode("11211")['Longitude']
res = search.by_coordinate(lat, lon, radius = 3)
reszip = []
for elem in res:
    reszip.append(elem["Zipcode"])
reszip

['11211', '11206', '11205', '11222', '10002']

In [60]:
pipeline = [
    { "$match" : { "facilityzipcode" : {"$in": reszip}} },
    {"$group": {"_id" : {"zipcode": "$facilityzipcode"}, 
                "total": {"$sum":1}}},
    {"$sort": {"total":-1}}
]
list(db.markets.aggregate(pipeline))

[{'_id': {'zipcode': '11206'}, 'total': 2},
 {'_id': {'zipcode': '11222'}, 'total': 1},
 {'_id': {'zipcode': '11205'}, 'total': 1}]

In [62]:
list(db.markets.find({"facilityzipcode" : '11206'}))

[{'_id': ObjectId('5a809ab5a4629b16115abd9c'),
  'facilityaddinfo': "Spend $5 in EBT and get a $2 Health Buck coupon.<br><br><b>Program Website:</b> <a  href=''http://http://grahamavenuebid.net/event/farmers-market-returns-to-cook-st-2'' target=''_blank''>http://http://grahamavenuebid.net/event/farmers-market-returns-to-cook-st-2</a>",
  'facilitycity': 'Brooklyn',
  'facilityname': "Graham Avenue Farmers' Market",
  'facilitystate': 'NY',
  'facilitystreetname': 'Ave of Puerto Rico',
  'facilityzipcode': '11206',
  'latitude': '41',
  'longitude': '-74'},
 {'_id': ObjectId('5a809ab5a4629b16115abdbe'),
  'facilityaddinfo': "Spend $5 in EBT and get a $2 Health Buck coupon.<br><br><b>Program Website:</b> <a  href=''http://www.harvesthomefm.org'' target=''_blank''>http://www.harvesthomefm.org</a>",
  'facilitycity': 'Brooklyn',
  'facilityname': "Harvest Home Marcy Park Farmers' Market",
  'facilitystate': 'NY',
  'facilitystreetname': 'Myrtle Ave',
  'facilityzipcode': '11206',
  'latitu

## Subway

Ces quartiers présentent l'avantage ou l'inconvénient d'être loin de l'agitation de Manhattan. J'aimerais pouvoir me déplacer facilement en métro. 

In [ ]:
def insert_data_subway(path, col):
    db[col].drop()
    json_data = open(path, 'r')
    parsed = json.loads(json_data.read())
    search = ZipcodeSearchEngine()
    for doc in parsed['data']:
        k = {}
        k['ObjectID'] = doc[8]
        k['url'] = doc[9]
        k['name'] = doc[10]
        geom = doc[11].replace('(', '').replace(')', '').split(' ')
        k['zipcode'] = search.by_coordinate(float(geom[2]), float(geom[1]), radius=3, returns= 1)[0]["Zipcode"]
        k['coord'] = [geom[2], geom[1]]
        k['line'] = doc[12]
        db[col].insert_one(k)
    del json_data
    del parsed

In [ ]:
insert_data_subway("subway.json", "subway2")

In [ ]:
db.subway2.find().count()

In [ ]:
db.subway2.find_one()

In [81]:
pipeline = [{"$match":{"$or" : [{"zipcode" : "11206"}, {"zipcode" : "11211"}]}},
            {"$group": {"_id" : {"line": "$line"}}}
           ]
list(db.subway2.aggregate(pipeline))

[{'_id': {'line': 'G'}},
 {'_id': {'line': 'J-M'}},
 {'_id': {'line': 'G-L'}},
 {'_id': {'line': 'J-M-Z'}},
 {'_id': {'line': 'L'}}]

Malheuresement il ne semblerait pas qu'une ligne soit directe vers l'agitation de Times Square (zone globalement dans le zipcode 10020). Parce qu'on ne peut trouver une localisation parfaite, je préfère sacrifier la facilité de transport vers Manhattan pour un quartier plus calme et correspondand à mes critères sur les piscines, marchés et restaurants.

In [108]:
pipeline = [{"$match":{"$or" : [{"zipcode" : "10020"}]}},
            {"$group": {"_id" : {"line": "$line"}}}
           ]
list(db.subway2.aggregate(pipeline))

[{'_id': {'line': 'A-C-E-N-Q-R-GS-1-2-3-7'}},
 {'_id': {'line': 'N-Q-R'}},
 {'_id': {'line': 'A-B-C-D-1'}},
 {'_id': {'line': 'B-D-E'}},
 {'_id': {'line': '1'}},
 {'_id': {'line': 'C-E'}}]

## Resultat final

A partir de trois critères : piscines, restaurants (français) et marchés/primeurs, mon choix se porte sur deux codes postaux voisins à Brooklyn : 11211 et 11206. Il s'agit finalement du quartier de Williamsbourg à Brooklyn.

Pour un peu mieux visualiser ce résultat, je voudrais les représenter sur une carte. Malheuresement, la base de données des marchés n'a pas de coordonnées spatiales précises. Pour faciliter ma visualisation j'ai (un peu) triché et regardé sur GoogleMap où se trouvaient les Graham Avenue Farmer's Market et Harverst Home Marcy Parket Farmer's Market.


 - Graham Avenue Farmer's Market : 40.704651, -73.942393
 - Harverst Home Marcy Park Farmer's Market : 40.700190, -73.909556 

In [ ]:
import folium
from folium.features import CustomIcon

icon_pool_url = 'https://cdn3.iconfinder.com/data/icons/hotel-services-facilities-2/256/Swimming_Pool-512.png'
icon_market_url = 'https://cdn3.iconfinder.com/data/icons/map-markers-1/512/supermarket-512.png'


res_pool = list(db.pools.find({'zipcode' : "11211"}))
res_market = [{'coord': [40.704651, -73.942393], 'name': "Graham Avenue Farmer Market"},
             {'coord': [40.700190, -73.909556], 'name': "Harverst Home Marcy Park Farmer Market"}]

m = folium.Map(location=[40.706150, -73.934228], zoom_start=14)
for r in res_pool:
    folium.Marker(r['average_coord'], popup=r['name'].replace("'", ""), icon=CustomIcon(icon_pool_url)).add_to(m)

for r in res_market:
    folium.Marker(r['coord'], popup=r['name'], icon=CustomIcon(icon_market_url)).add_to(m)
    
    
m